# The Real code

In [1]:
using Random, Distributions, Plots, XLSX

What needs to be done:
. Statistics along the way
. the simulation gives a shit ton of unfullfilled values, seems fucked!
. Last simulation
... Plots


Questions
. Weird arrival at midnight
. Why the large amount of unfullfilled values?
. Should we keep values for every step at every run or just the final statistics at every run?


In [2]:
### Initial functions to load lambda and mu from the ArrivalProfiles.xlsx

# Change the type to Float64 from Int and String for the loaded vectors
function type_change(v)
    v_ny = Vector{Float64}()
    for i in 1:length(v)
        if typeof(v[i]) == String
            push!(v_ny,parse(Float64,v[i]))
        elseif typeof(v[i]) == Int64
            push!(v_ny,float(v[i]))
        else
            push!(v_ny,v[i])
        end
    end
    return v_ny
end


function load_arri(path="C:/Users/rasmu/Desktop/OneDrive/OneDrive - Danmarks Tekniske Universitet/Decisions under uncertainty/Projekt/ArrivalProfiles.xlsx")
    xf = XLSX.readxlsx(path) # Loading

    # Lambda is given as expected arrivals therefore inverse
    lambda_port = 1 ./ type_change(xf[1]["C2:AA2"])
    lambda_dtu = 1 ./ type_change(xf[1]["C3:AA3"])

    # Travel time
    mu_rammer_dtu = type_change(xf[1]["C7:AA7"])
    mu_rammer_port = type_change(xf[1]["C8:AA8"])

    return lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port
end


# Might not be just as fast as PriorityQueue but it is not that much behind
# Inserts a value at the spot it fits
function insert_and_sort!(vect::Vector, x)
    (splice!(vect, searchsorted(vect,x), [x]); vect) 
end


insert_and_sort! (generic function with 1 method)

In [3]:
struct Environment # The environment including arrival rates and travel rates
    lambda_dtu::Vector{Float64}  #arrival rate 1
    mu_rammer_dtu::Vector{Float64} #service rate 1
    lambda_port::Vector{Float64} # arrival rate 2
    mu_rammer_port::Vector{Float64} #service rate 2
    
    # loading the vectors from excel
    lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port = load_arri() # NEEDS the path to load the document from!
    Environment() = new(lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port)
end

mutable struct Stats # Vector including [Station 1, Station 2]
    n_cars::Vector{Int64} # Number of cars at the stations at any given time
    un_fullfilled::Vector{Int64} # Amount of times a customer arrives to an empty station
    fullfilled::Vector{Int64} # Amount of times a costumer arrives and gets a car
    empty_time::Vector{Float64} # The time any given station has 0 cars
    currently_empty::Vector{Bool} # Is the station empty
end

struct Balance
    desired::Array{Tuple{Int64,Int64},1}
    rebalance_times::Vector{Float64}
end


In [4]:

function time_change(e::Array{Tuple{Float64,String},1},k::Int64,day::Int64) # problems
    # Switch day and reset index
    if k == 25
        k = 0
        day += 25
    end
    # Insert the next time change
    popfirst!(e)[1]
    insert_and_sort!(e,(k+day,"time_change"))

    # Change index for time
    k += 1
    return k,day
end

function arrival_student(station::Int64,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats,k::Int64)
    
    # Insert next arrival based on previous arrival
    if station == 1
        insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k]))+e[1][1],"st1"))
    elseif station == 2
        insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k]))+e[1][1],"st2"))
    end

    if stat.currently_empty[station] == true # Student arrives at empty station. That is one unsatisfied costumer :(
        stat.un_fullfilled[station] += 1
        popfirst!(e)
        
    else # Costumer leaves station
        stat.n_cars[station] -= 1 # removing car from station
        stat.fullfilled[station] += 1 # one fullfilled request
        
        # Insert travel time based on arrival station
        if station == 1
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2")) 
        elseif station == 2
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st1")) 
        end

        if stat.n_cars[station] == 0 # station becomes empty
            stat.empty_time[station] -= popfirst!(e)[1] # Empty station timer starts 
            stat.currently_empty[station] = true
        else
            popfirst!(e)
        end
    end 

end 

function arrival_vehicle(station::Int64,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats)

    if stat.currently_empty[station] == true # Arriving at an empty station
        stat.n_cars[station] += 1 # arrive
        stat.empty_time[station] += popfirst!(e)[1] # End empty timer
        stat.currently_empty[station] = false
    else
        stat.n_cars[station] += 1 # arrive
        popfirst!(e)
    end

end


function rebalancing(rebal::Balance,stat::Stats,env::Environment,e::Array{Tuple{Float64,String},1},k::Int64,rebalance_action::Int64)
    # NEEDS Documentation and a serious debugging
    if stat.n_cars[2] >= rebal.desired[rebalance_action][1]
        for i in 1:rebal.desired[rebalance_action][1]
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_dtu[k]))+e[1][1],"que_st1")) 
            stat.n_cars[2] -= 1
        end
        
        # Start empty timer
        if stat.n_cars[2] == 0
            stat.empty_time[2] -= e[1][1]
            stat.currently_empty[2] = true
        end

    else
        for i in 1:stat.n_cars[2]
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_dtu[k]))+e[1][1],"que_st1"))
            stat.n_cars[2] -= 1
        end
        if stat.currently_empty[2] == false
            stat.empty_time[2] -= e[1][1]
            stat.currently_empty[2] = true
        end
    end

    # For station 1
    if stat.n_cars[1] >= rebal.desired[rebalance_action][2]
        for i in 1:rebal.desired[rebalance_action][2]
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2"))
            stat.n_cars[1] -= 1
        end
        
        # Start empty timer
        if stat.n_cars[1] == 0
            stat.empty_time[1] -= e[1][1]
            stat.currently_empty[1] = true
        end
    else
        for i in 1:stat.n_cars[1]
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2"))
            stat.n_cars[1] -= 1
        end
        if stat.currently_empty[1] == false
            stat.empty_time[1] -= e[1][1]
            stat.currently_empty[1] = true
        end
    end 

    insert_and_sort!(e,(24.0+popfirst!(e)[1],"rebalance"))

    # Virkelig grim måde at gøre det på 
    if rebalance_action == length(rebal.desired)
        rebalance_action = 1
    else
        rebalance_action += 1
    end
    
end



rebalancing (generic function with 1 method)

In [5]:
function sim(days_to_run::Int64,initial_n_cars::Vector{Int64}, rebal::Balance, env::Environment)

    ###########
    # Initial Misc
    stat = Stats([initial_n_cars[1],initial_n_cars[2]],[0,0],[0,0],[0,0],[false,false])
    e = Array{Tuple{Float64,String},1}()
    runs = 0
    
    ###########
    # arrival and travel rates
    insert_and_sort!(e,(0.0,"time_change")) # THIS NEEDS TO BE FIXED!!!!!!!!
    k = 1
    day = 0
    
    ###########
    # Arrivals
    insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k])),"st1"))
    insert_and_sort!(e,(rand(Exponential(env.lambda_port[k])),"st2"))
    
    ###########
    # Rebalance
    rebalance_action = 1
    for i in 1:length(rebal.rebalance_times)
        insert_and_sort!(e,(rebal.rebalance_times[i],"rebalance"))
    end


    while e[1][1] <= days_to_run*24.0 && runs <= 100000
        load_type = e[1][2]
        if load_type == "st1"
            arrival_student(1,e,env,stat,k)
        elseif load_type == "st2"
            arrival_student(2,e,env,stat,k)
        elseif load_type == "que_st1"
            arrival_vehicle(1,e,env,stat)
        elseif load_type == "que_st2"
            arrival_vehicle(2,e,env,stat)
        elseif load_type == "time_change"
            k,day = time_change(e,k,day)
        elseif load_type == "rebalance"
            rebalancing(rebal,stat,env,e,k,rebalance_action)
        else
            throw(ErrorException("Issue in the event type!"))
        end
        runs += 1 # Checks the amount of events
    end
    
    # Make sure the empty timer stops at the end of the run
    if stat.currently_empty[1] == true
        stat.empty_time[1] += e[1][1]
    elseif stat.currently_empty[2] == true
        stat.empty_time[2] += e[1][1]
    end
    
    return stat.n_cars,stat.un_fullfilled,stat.fullfilled,stat.empty_time,runs
end

sim (generic function with 1 method)

### Testing

In [6]:

function main(initial_n_cars::Vector{Int64},days_to_run::Int64,n_of_runs::Int64,rebalance::Tuple{Vector{Tuple{Int64, Int64}}, Vector{Float64}},path = "C:/Users/rasmu/Desktop/OneDrive/OneDrive - Danmarks Tekniske Universitet/Decisions under uncertainty/Projekt/ArrivalProfiles.xlsx")
    env = Environment()
    rebal = Balance(rebalance[1],rebalance[2])
    overview = []
    efficiancy = []

    for i in 1:n_of_runs
        a = sim(days_to_run,initial_n_cars,rebal,env)
        overview = [overview;a]
        # NEeds final stats
    
        efficiancy_fullfilled = [a[3][1]/(a[3][1]+a[2][1]), a[3][2]/(a[3][2]+a[2][2]) ]
        efficiancy_emptytime = [a[4][1]/(days_to_run*24),a[4][2]/(days_to_run*24)]

        # print("Efficiancy percentage for station 1 is ")
        # printstyled(round(efficiancy_fullfilled[1]*100,digits=2),"%"; color=:blue)
        # print(" and ")
        # printstyled(round(efficiancy_fullfilled[2]*100,digits=2),"%"; color=:blue)
        # print(" for station 2. Percentage of time spent empty for station 1 is ")
        # printstyled(round(efficiancy_emptytime[1]*100,digits=2),"%"; color=:blue)
        # print(" and ")
        # printstyled(round(efficiancy_emptytime[2]*100,digits=2),"%"; color=:blue)
        # println(" for station 2.")

        efficiancy = [efficiancy; [efficiancy_fullfilled]]
    end
    
    return overview, efficiancy
end

main (generic function with 2 methods)

In [14]:
########### User manual ############
##### Inputs: main(1,2,3,4)
# 1: [initial cars at station 1, initial cars at station 2] (Int,Int)
# 2: days to run each simulation (Int)
# 3: Amount simulations to run (Int)
# 4: ( [(Desired amount at station 1 for first reblance, station 2 at first rebalance), (station 1 at second, station 2 at second), ..., (station 1 at n rebalance, station 2 at n)], [time of first rebalance, ..., time at n event] ) (Tuple(Int,Int),Float)



main([60,60],100,20,([(100,20),(30,90)],[2.0,13.0]))

Efficiancy percentage for station 1 is 81.99% and 60.87% for station 2. Percentage of time spent empty for station 1 is 5.9% and 10.53% for station 2.
Efficiancy percentage for station 1 is 85.84% and 55.28% for station 2. Percentage of time spent empty for station 1 is 3.52% and 14.34% for station 2.
Efficiancy percentage for station 1 is 86.08% and 55.78% for station 2. Percentage of time spent empty for station 1 is 3.78% and 12.5% for station 2.
Efficiancy percentage for station 1 is 83.57% and 61.42% for station 2. Percentage of time spent empty for station 1 is 4.33% and 11.59% for station 2.
Efficiancy percentage for station 1 is 88.19% and 59.18% for station 2. Percentage of time spent empty for station 1 is 2.54% and 14.21% for station 2.
Efficiancy percentage for station 1 is 85.19% and 56.99% for station 2. Percentage of time spent empty for station 1 is 3.75% and 13.11% for station 2.
Efficiancy percentage for station 1 is 80.07% and 58.84% for station 2. Percentage of time

(Any[([119, 0], [2201, 3736], [10018, 5812], [141.5367453153267, 252.78903834551556], 51331), ([91, 25], [1436, 4912], [8706, 6073], [84.52343190196689, 344.25011031665554], 48349), ([36, 78], [1313, 4026], [8121, 5079], [90.80062803471174, 300.11549904143885], 44482), ([82, 35], [1631, 3736], [8296, 5948], [103.92877086836279, 278.0685396378526], 45992), ([111, 0], [1118, 4401], [8352, 6381], [61.023901160119294, 341.00956189101635], 46986), ([89, 30], [1503, 4142], [8645, 5489], [90.0825099056674, 314.7408909759388], 46853), ([113, 0], [2214, 3521], [8895, 5034], [140.33002370586405, 277.4468192332756], 46874), ([64, 49], [1285, 4112], [8760, 5710], [71.14316413554025, 296.4341579228476], 47218), ([64, 54], [1275, 5029], [7780, 5441], [91.80290779708093, 379.23584855130434], 44782), ([77, 36], [1100, 3724], [8522, 6499], [67.83100742221905, 305.3251579747316], 46950), ([90, 28], [1756, 4866], [8169, 5399], [105.45054084227922, 363.27583357023786], 46279), ([102, 15], [2074, 3461], [9

In [9]:
# Tjekker start distribution, virker som om den er stortset irrelevant

effic_st1 = []
effic_st2 = []
position = []
for n in 60:60
    for i in 1:(n-1)
        _, effic = main([i,n-i],100,n, ([(100,20),(40,80)],[7.0,13.0]) )
        total_effic = [effic[1][1],effic[1][2]]
        for j in 2:n
            total_effic = [effic[j][1]+total_effic[1],effic[j][2]+total_effic[2]]
        end
        
        push!(effic_st1,total_effic[1]/n)
        push!(effic_st2,total_effic[2]/n)
        push!(position,[i,n-i,n])

    end
end

println(findmax(effic_st1)," for station1")
println(findmax(effic_st2)," for station2")
println(position[findmax(effic_st1)[2]]," for station1")
println(position[findmax(effic_st2)[2]]," for station2")


(0.687261116199373, 26) for station1
(0.5760346712843007, 39) for station2
[26, 34, 60] for station1
[39, 21, 60] for station2


In [8]:

effic_st1 = []
effic_st2 = []
position = []
for n in 90:90
    for i in 1:20:n*2
        for j in 1:20:n*2
            _, effic = main([n,n],100,n, ([(i,n*2-i),(j,n*2-j)],[7.0,13.0]) )
            total_effic = [effic[1][1],effic[1][2]]
            for p in 2:n-1
                total_effic = [effic[p][1]+total_effic[1],effic[p][2]+total_effic[2]]
            end
            
            push!(effic_st1,total_effic[1]/n)
            push!(effic_st2,total_effic[2]/n)
            push!(position,[[i,n*2-i],[j,n*2-j]])
        end
    end
end

println(findmax(effic_st1)," for station1")
println(findmax(effic_st2)," for station2")
println(position[findmax(effic_st1)[2]]," for station1")
println(position[findmax(effic_st2)[2]]," for station2")


(0.9463146026522803, 78) for station1
(0.9675967002458218, 4) for station2
[[161, 19], [101, 79]] for station1
[[1, 179], [61, 119]] for station2


In [12]:
effic_st1

36-element Vector{Any}:
 0.22061520491745754
 0.21459199049414773
 0.21922134138184335
 0.2194600646789736
 0.22352523167697702
 0.21922787208378686
 0.29066320348823843
 0.29097117847790643
 0.2908920540826493
 0.2884305638016938
 ⋮
 0.3959363332897391
 0.39706269454261106
 0.39973728084852855
 0.4203153236034081
 0.4183310782258446
 0.4211362321132467
 0.4216330892827385
 0.4212648239171392
 0.4178566291772772